## This is an example showcasing the loading of trARPES data as collected using the Phoibos detector at FHI Berlin.
The band dispersion is loaded as a xarray dataframe following a conversion to the [NeXus format](https://manual.nexusformat.org/classes/contributed_definitions/NXmpes.html#nxmpes) using the [Nomad Parser Nexus](https://github.com/nomad-coe/nomad-parser-nexus).

First, the SpecsScan class is imported which has the scan loader as its class method.

In [ ]:
%load_ext autoreload
%autoreload 2
from specsscan import SpecsScan
from pathlib import Path

Here, a SpecsScan class instance is created as per the configuration provided in [config.yaml](../tests/data/config.yaml). The user may set the entries in config.yaml file, for example, the data path and conversion parameters as per the requirements before creating this instance

### The path may changed to point to the scan folder of the data of interest (for example, on a server drive)

In [ ]:
metadata = {}
# manual Meta data. These should ideally come from an Electronic Lab Notebook.
#General
metadata['experiment_summary'] = 'TbTe3 tilt map around EF with 800nm pump at 20fs after pump-probe overlap'
metadata['entry_title'] = 'TbTe3 XUV Fermi surface map at 20 fs'
metadata['experiment_title'] = 'TbTe3 XUV Fermi surface map at 20 fs'

#User
# Fill general parameters of NXuser
# TODO: discuss how to deal with multiple users?
metadata['user0'] = {}
metadata['user0']['name'] = 'Laurenz Rettig'
metadata['user0']['role'] = 'Principal Investigator'
metadata['user0']['affiliation'] = 'Fritz Haber Institute of the Max Planck Society'
metadata['user0']['address'] = 'Faradayweg 4-6, 14195 Berlin'
metadata['user0']['email'] = 'rettig@fhi-berlin.mpg.de'

metadata['user1'] = {}
metadata['user1']['name'] = 'William Windsor'
metadata['user1']['role'] = 'Principal Investigator'
metadata['user1']['affiliation'] = 'Fritz Haber Institute of the Max Planck Society'
metadata['user1']['address'] = 'Faradayweg 4-6, 14195 Berlin'
metadata['user1']['email'] = 'windsor@fhi-berlin.mpg.de'

metadata['instrument'] = {}
# energy resolution
metadata['instrument']['energy_resolution'] = 150.
metadata['instrument']['electronanalyser'] = {}
metadata['instrument']['electronanalyser']['energy_resolution'] = 120
metadata['instrument']['electronanalyser']['angular_resolution'] = 0.2
metadata['instrument']['electronanalyser']['spatial_resolution'] = 0.5

#probe beam
metadata['instrument']['beam']={}
metadata['instrument']['beam']['probe']={}
metadata['instrument']['beam']['probe']['incident_energy'] = 21.7
metadata['instrument']['beam']['probe']['incident_energy_spread'] = 0.11
metadata['instrument']['beam']['probe']['pulse_duration'] = 20.
metadata['instrument']['beam']['probe']['frequency'] = 500.
metadata['instrument']['beam']['probe']['incident_polarization'] = [1, 1, 0, 0] # p pol Stokes vector
metadata['instrument']['beam']['probe']['extent'] = [80., 80.]
#pump beam
metadata['instrument']['beam']['pump']={}
metadata['instrument']['beam']['pump']['incident_energy'] = 1.55
metadata['instrument']['beam']['pump']['incident_energy_spread'] = 0.08
metadata['instrument']['beam']['pump']['pulse_duration'] = 35.
metadata['instrument']['beam']['pump']['frequency'] = 500.
metadata['instrument']['beam']['pump']['incident_polarization'] = [1, -1, 0, 0] # s pol Stokes vector
metadata['instrument']['beam']['pump']['incident_wavelength'] = 800.
metadata['instrument']['beam']['pump']['average_power'] = 224.
metadata['instrument']['beam']['pump']['pulse_energy'] = metadata['instrument']['beam']['pump']['average_power']/metadata['instrument']['beam']['pump']['frequency']#µJ
metadata['instrument']['beam']['pump']['extent'] = [300/4*2.34, 270/4*2.35] #Gaussian 4sigma -> FWHM
metadata['instrument']['beam']['pump']['fluence'] = 1.00
metadata['instrument']['beam']['pump']['delay'] = 0.02

#sample
metadata['sample']={}
metadata['sample']['preparation_date'] = '2017-03-19T10:00:00+00:00'
metadata['sample']['preparation_description'] = 'Cleaved'
metadata['sample']['sample_history'] = 'Cleaved in UHV'
metadata['sample']['chemical_formula'] = 'TbTe3'
metadata['sample']['description'] = 'cleaved single crystal of TbTe3'
metadata['sample']['name'] = 'TbTe3 Single Crystal'

#metadata["scan_info"] = {}
#metadata["scan_info"]["trARPES:XGS600:PressureAC:P_RD"] = 2.5E-11
#metadata["scan_info"]["trARPES:Carving:TEMP_RBV"] = 70


In [ ]:
config = {"nexus":{"definition": "NXmpes"}, "spa_params":{"crop":True, "ek_min":20.2, "ek_max":22.5, "ang_min":-13.5, "ang_max":13.5}}

In [ ]:
sps = SpecsScan(config=config, user_config="../specsscan/config/example_config_FHI.yaml")
path = "../tests/data/" # Path to the test data set

The load_scan method loads the scan as an xarray along with the metadata needed for nexus conversion. The progress bars can be activated by changing the config parameter, enable_nested_progress_bar, to true in config.yaml 

In [ ]:
res_xarray = sps.load_scan(
    scan=6833, # Tilt scan
    metadata=metadata,
)

In [ ]:
%matplotlib inline
res_xarray[:,29,:].plot()

In [ ]:
%matplotlib inline
res_xarray.loc[{"energy":slice(21.6, 21.8)}].sum(axis=2).plot()

In [ ]:
sps.save("FS_map_CsV3Sb5.h5")

The data may be visualised as a 2D plot by slicing the xarray in 2 dimensions keeping the third fixed. In the plot below, the Angle vs Ekin data is plotted for a fixed mirrorX value.

In [ ]:
res_xarray.dims

In [ ]:
sps.save("FS_map_CsV3Sb5.nxs")

The metadata associated with the scan is added as an attribute to the xarray

3D scans, where the images are recorded as a function of a third parameter (generally delay or in this case, mirrorX), can also be loaded with an option to average only the given iterations passed as a list or slice object. 

In [ ]:
from jupyterlab_h5web import H5Web

In [ ]:
H5Web('spectest.mpes.nxs')